<a href="https://colab.research.google.com/github/Tom-Jung/ML_Colab/blob/main/TF_2_x_LEC_31_Brain_Tumor_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as rf
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root dir
data root dir
train data root dir
test data root dir

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
import os

ROOT_DIR = '/content'
DATA_ROOT_DIR = os.path.join(ROOT_DIR, 'brain_tumor')
TRAIN_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, 'Training')
TEST_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, 'Testing')

# Google Drive 마운트해서 데이터셋 가져옴

In [30]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [31]:
import shutil
import os

try:
    dataset_path = '/content/gdrive/My Drive/Colab Notebooks/dataset'
    
    shutil.copy(os.path.join(dataset_path, 'brain_tumors.zip'), '/content')

except Exception as err:
    print(str(err))

In [32]:
import os
import shutil

if os.path.exists(DATA_ROOT_DIR):
   shutil.rmtree(DATA_ROOT_DIR)
   print(DATA_ROOT_DIR + ' is removed !!!')

In [37]:
# 압축파일 풀기

import zipfile
with zipfile.ZipFile(os.path.join(ROOT_DIR, 'brain_tumors.zip'), 'r') as target_file:
  target_file.extractall(DATA_ROOT_DIR)

brain_tumor/Testing 확인
brain_tumor/Training 확인

In [43]:
import os

#========================================================
# train 정답 및 전체 데이터 개수 확인
#========================================================

train_label_name_list = os.listdir(TRAIN_DATA_ROOT_DIR)

print(train_label_name_list)

for label_name in train_label_name_list:

    print('train label : ', label_name,' => ', len(os.listdir(os.path.join(TRAIN_DATA_ROOT_DIR, label_name))))

print('=====================================================')

#========================================================
# test 정답 및 전체 데이터 개수 확인
#========================================================

test_label_name_list = os.listdir(TEST_DATA_ROOT_DIR)

print(test_label_name_list)

for label_name in test_label_name_list:

    print('test label : ', label_name, ' => ', len(os.listdir(os.path.join(TEST_DATA_ROOT_DIR, label_name))))

print('=====================================================')


['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
train label :  glioma_tumor  =>  826
train label :  meningioma_tumor  =>  822
train label :  no_tumor  =>  395
train label :  pituitary_tumor  =>  827
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
test label :  glioma_tumor  =>  100
test label :  meningioma_tumor  =>  115
test label :  no_tumor  =>  105
test label :  pituitary_tumor  =>  74


## Testing 디렉토리 모든 파일을 Training 동일 디렉토리로 move

In [53]:
test_label_name_list = os.listdir(TEST_DATA_ROOT_DIR)
print(test_label_name_list)

for label_name in test_label_name_list: # Testing 디렉토리 각각의 Label에서 실행
     test_path = os.path.join(TEST_DATA_ROOT_DIR, label_name)
     test_file_list = os.listdir(test_path)
     train_path = os.path.join(TRAIN_DATA_ROOT_DIR, label_name)
     move_num =0
     for test_image_file in test_file_list: # Testing 모든 data를 Training 디렉토리로 이동
          shutil.move(os.path.join(test_path, test_image_file),
                          os. path.join(train_path, test_image_file))
          move_num=move_num+1

     print(str(move_num) + 'is move to ' + label_name +'(Testing => Trainning) ')

['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
100is move to glioma_tumor(Testing => Trainning) 
115is move to meningioma_tumor(Testing => Trainning) 
105is move to no_tumor(Testing => Trainning) 
74is move to pituitary_tumor(Testing => Trainning) 


# move 완료된 train 전체 데이터 개수 확인

In [59]:
#========================================================
# train 정답 및 전체 데이터 개수 확인
#========================================================

train_label_name_list = os.listdir(TRAIN_DATA_ROOT_DIR)
print(train_label_name_list)
for label_name in train_label_name_list:
    print('train label : ', label_name, '=>', len(os.listdir(os.path.join(TRAIN_DATA_ROOT_DIR, label_name))))

print('=========================================================')

['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
train label :  glioma_tumor => 926
train label :  meningioma_tumor => 937
train label :  no_tumor => 396
train label :  pituitary_tumor => 901


# Move 비율 설정

In [ ]:
import os
import shutil
import random

MOVE_RATIO = 0.2  # train:test = 80:20, 즉 train 데이터 20%를 test 데이터로 사용

# 파일 move train_data_dir => test_data_dir

label_name_list = os.listdir(TRAIN_DATA_ROOT_DIR)
for label_name in label_name_list:
  #파일 move 하기 위한 src_dir_path, dst_dir_path 설정
  #==================================
    src_dir_path = os.path.join(TRAIN_DATA_ROOT_DIR, label_name)
    dst_dir_path = os.path.join(TEST_DATA_ROOT_DIR, label_name)
  #==================================
     train_data_file_list = os.listdir(src_dir_path)
     random.shuffle(train_data_file_list)   # data shuffle
     split_num=int(MOVE_RATIO*len(train_data_file_list))
     test_data_file_list=train_data_file_list[0:split_num]
     for test_data_file in test_data_file_list:
       try:
         shutil.move(os.path.join(src_dir_path, test_data_file),
                         os.path.join(dst_dir_path))